In [23]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
from scipy import stats
import math
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import pandas as pd
import geopandas as gpd

In [24]:
def custom_dist(u, v):
    if u[2] == v[2]:
        return sys.float_info.max
    else:
        return haversine([u[0], u[1]], [v[0], v[1]])

thresholds = {'CurbRamp': 0.0075,
                      'NoCurbRamp': 0.0075,
                      'SurfaceProblem': 0.01,
                      'Obstacle': 0.01,
                      'NoSidewalk': 0.01,
                      'Crosswalk': 0.01,
                      'Signal': 0.01,
                      'Occlusion': 0.01,
                      'Other': 0.01,
                      'Problem': 0.01}

def cluster(labels, label_type, thresholds):

    # Makes a normal dist matrix for a single user, but uses special dist function for multi-user clustering that
    # prevents the same user's attributes from being clustered together.
    dist_matrix = pdist(np.array(labels[['lat', 'lng', 'user_id']].values), custom_dist)
    link = linkage(dist_matrix, method='complete')
    curr_type = labels['label_type'].values

    # Copies the labels dataframe and adds a column to it for the cluster id each label is in.
    labelsCopy = labels.copy()
    labelsCopy.loc[:,'cluster_id'] = fcluster(link, t=thresholds[label_type], criterion='distance')

    # Cuts tree so that only labels less than clust_threth kilometers apart are clustered.
    clusters = labelsCopy.groupby('cluster_id')

    # Computes the center of each cluster and assigns temporariness and severity.
    cluster_list = [] # list of tuples (label_type, cluster_num, lat, lng, severity, temporary).
    for clust_num, clust in clusters:
        #ave_pos = np.mean(clust['coords'].tolist(), axis=0) # use ave pos of clusters.
        #ave_sev = None if pd.isnull(clust['severity']).all() else int(round(np.median(clust['severity'][~np.isnan(clust['severity'])])))
        #ave_temp = None if pd.isnull(clust['temporary']).all() else bool(round(np.mean(clust['temporary'])))

        cluster_list.append((curr_type, clust_num))

    cluster_df = pd.DataFrame(cluster_list, columns=['label_type', 'cluster_id'])

    return (cluster_df, labelsCopy)


# Pick which label types should be included in clustering, and which should be included in the "Problem" type.
label_types = ['CurbRamp', 'NoSidewalk', 'Occlusion', 'SurfaceProblem', 'Obstacle', 'Other', 'NoCurbRamp', 'Crosswalk', 'Signal']
# problem_types = ['SurfaceProblem', 'Obstacle']

# These are the columns required in the POST requests for the labels and clusters, respectively.
label_cols = ['label_id', 'label_type', 'cluster_id']
cluster_cols = ['label_type', 'cluster_id', 'lat', 'lng', 'severity', 'temporary']


def cluster_label_type_at_index(label_data,i):
        clusters_for_type_i = pd.DataFrame(columns=cluster_cols)
        labels_for_type_i = pd.DataFrame(columns=label_cols)

        label_type = label_types[i]
        # if label_type == 'Problem':
        #     type_data = label_data[label_data.label_type.isin(problem_types)]
        # else:
        type_data = label_data[label_data.label_type == label_type]

        # If there are >1 labels, we can do clustering. Otherwise just copy the 1 (or 0) labels.
        if type_data.shape[0] > 1:
            (clusters_for_type_i, labels_for_type_i) = cluster(type_data, label_type, thresholds)
        elif type_data.shape[0] == 1:
            labels_for_type_i = type_data.copy()
            labels_for_type_i.loc[:,'cluster_id'] = 1 # Gives the single cluster a cluster_id of 1.
            labels_for_type_i.loc[:,'label_type'] = label_type # Gives Problem type if needed.
            clusters_for_type_i = labels_for_type_i.filter(items=cluster_cols)

        return (label_type, clusters_for_type_i, labels_for_type_i)

In [25]:
df = gpd.read_file('data/attributesWithLabels_q01.json')

In [26]:
df

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,label_date,label_severity,label_is_temporary,agree_count,disagree_count,notsure_count,label_tags,label_description,user_id,geometry
0,68021523,CurbRamp,17408,169473536,Belltown,1.0,False,33,iKQUgAy3Qng_G5aAZrrAqQ,82.0625,...,2019-03-12 21:42:51.474,1.0,False,1,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34361 47.61735)
1,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,34,iKQUgAy3Qng_G5aAZrrAqQ,71.5625,...,2019-03-12 21:42:55.317,1.0,False,3,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736)
2,68021515,CurbRamp,17408,169473536,Belltown,1.0,False,35,iKQUgAy3Qng_G5aAZrrAqQ,332.0000,...,2019-03-12 21:42:58.830,1.0,False,0,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34375 47.61745)
3,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,36,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,2019-03-12 21:43:12.914,1.0,False,2,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736)
4,68021306,CurbRamp,25604,484571797,Belltown,1.0,False,37,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,2019-03-12 21:43:14.595,1.0,False,4,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34355 47.61740)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58164,67967728,CurbRamp,11675,9266258,Crown Hill,NaN,False,232653,PlClKXA8ouVEUv33dO0DAA,6.0000,...,2023-01-30 19:06:47.670,NaN,False,0,0,0,[ ],None,37a61b91-2d4c-4cc5-b80a-8615d0311fc1,POINT (-122.37270 47.69652)
58165,68019117,NoSidewalk,7420,6467025,North Beach/Blue Ridge,3.0,False,233013,WV4BznnqrtclwL6hna8auA,281.7500,...,2023-02-05 13:58:37.022,3.0,False,0,0,0,"[ ""street has no sidewalks"" ]",None,6567cf55-9ecb-4c6c-962b-3a3cf9395718,POINT (-122.39974 47.69600)
58166,68018901,NoSidewalk,5351,6432939,North Beach/Blue Ridge,5.0,False,233302,UCWnkU_o7ODPwVQY4orXqQ,341.1250,...,2023-02-09 01:02:02.175,5.0,False,0,0,0,"[ ""street has no sidewalks"" ]",None,01f22e7e-868d-48be-ae5c-0440e94ace6b,POINT (-122.39241 47.69797)
58167,68018887,NoSidewalk,5351,6432939,North Beach/Blue Ridge,5.0,False,233303,Ci6d1cAqbWlzeAhTs5c3cQ,8.5000,...,2023-02-09 01:02:18.548,5.0,False,0,0,0,"[ ""street has no sidewalks"" ]",None,01f22e7e-868d-48be-ae5c-0440e94ace6b,POINT (-122.39226 47.69805)


In [27]:
# break geometry into lat and lng
df['lat'] = df.geometry.y
df['lng'] = df.geometry.x

In [28]:
df0 = cluster_label_type_at_index(df,0)[2]

In [29]:
df1 = cluster_label_type_at_index(df,1)[2]

In [30]:
df2 = cluster_label_type_at_index(df,2)[2]

In [31]:
df3 = cluster_label_type_at_index(df,3)[2]

In [32]:
df4 = cluster_label_type_at_index(df,4)[2]

In [33]:
df5 = cluster_label_type_at_index(df,5)[2]

In [34]:
df6 = cluster_label_type_at_index(df,6)[2]

In [35]:
df7 = cluster_label_type_at_index(df,7)[2]

In [36]:
df8 = cluster_label_type_at_index(df,8)[2]

In [37]:
df0["clustered"] = df0.duplicated(subset="cluster_id", keep=False)
df1["clustered"] = df1.duplicated(subset="cluster_id", keep=False)
df2["clustered"] = df2.duplicated(subset="cluster_id", keep=False)
df3["clustered"] = df3.duplicated(subset="cluster_id", keep=False)
df4["clustered"] = df4.duplicated(subset="cluster_id", keep=False)
df5["clustered"] = df5.duplicated(subset="cluster_id", keep=False)
df6["clustered"] = df6.duplicated(subset="cluster_id", keep=False)
df7["clustered"] = df7.duplicated(subset="cluster_id", keep=False)
df8["clustered"] = df8.duplicated(subset="cluster_id", keep=False)

In [38]:
df0

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,disagree_count,notsure_count,label_tags,label_description,user_id,geometry,lat,lng,cluster_id,clustered
0,68021523,CurbRamp,17408,169473536,Belltown,1.0,False,33,iKQUgAy3Qng_G5aAZrrAqQ,82.0625,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34361 47.61735),47.617348,-122.343605,9590,False
1,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,34,iKQUgAy3Qng_G5aAZrrAqQ,71.5625,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736),47.617363,-122.343590,12294,True
2,68021515,CurbRamp,17408,169473536,Belltown,1.0,False,35,iKQUgAy3Qng_G5aAZrrAqQ,332.0000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34375 47.61745),47.617451,-122.343750,6809,True
3,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,36,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736),47.617363,-122.343590,12693,True
4,68021306,CurbRamp,25604,484571797,Belltown,1.0,False,37,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34355 47.61740),47.617401,-122.343552,5826,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58158,67967734,CurbRamp,11675,9266258,Crown Hill,3.0,False,232647,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,37a61b91-2d4c-4cc5-b80a-8615d0311fc1,POINT (-122.37262 47.69654),47.696545,-122.372620,46,True
58159,67967740,CurbRamp,11675,9266258,Crown Hill,1.0,False,232648,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,37a61b91-2d4c-4cc5-b80a-8615d0311fc1,POINT (-122.37273 47.69658),47.696575,-122.372726,38,False
58160,67967724,CurbRamp,11675,9266258,Crown Hill,2.0,False,232649,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,37a61b91-2d4c-4cc5-b80a-8615d0311fc1,POINT (-122.37263 47.69650),47.696503,-122.372635,28,True
58164,67967728,CurbRamp,11675,9266258,Crown Hill,NaN,False,232653,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,37a61b91-2d4c-4cc5-b80a-8615d0311fc1,POINT (-122.37270 47.69652),47.696518,-122.372696,16,False


In [39]:
#concatenate all dataframes
clusters_q1 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8])

In [40]:
clusters_q1

,attribute_id,label_type,street_edge_id,osm_street_id,neighborhood,severity,is_temporary,label_id,gsv_panorama_id,heading,...,disagree_count,notsure_count,label_tags,label_description,user_id,geometry,lat,lng,cluster_id,clustered
0,68021523,CurbRamp,17408,169473536,Belltown,1.0,False,33,iKQUgAy3Qng_G5aAZrrAqQ,82.0625,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34361 47.61735),47.617348,-122.343605,9590,False
1,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,34,iKQUgAy3Qng_G5aAZrrAqQ,71.5625,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736),47.617363,-122.343590,12294,True
2,68021515,CurbRamp,17408,169473536,Belltown,1.0,False,35,iKQUgAy3Qng_G5aAZrrAqQ,332.0000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34375 47.61745),47.617451,-122.343750,6809,True
3,68021512,CurbRamp,17408,169473536,Belltown,1.0,False,36,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34359 47.61736),47.617363,-122.343590,12693,True
4,68021306,CurbRamp,25604,484571797,Belltown,1.0,False,37,gyFuxqdBmrjXpxMkvJLUnA,102.5000,...,0,0,[ ],None,fa0bbee4-b98d-4eb3-b6da-d9e9aa287f6c,POINT (-122.34355 47.61740),47.617401,-122.343552,5826,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58057,67969783,Signal,11675,9266258,Crown Hill,NaN,False,231079,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,28c0b06a-5d3f-4fb8-8341-575c1859f08a,POINT (-122.37266 47.69650),47.696499,-122.372658,11,False
58058,67969787,Signal,11675,9266258,Crown Hill,NaN,False,231080,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,28c0b06a-5d3f-4fb8-8341-575c1859f08a,POINT (-122.37267 47.69656),47.696560,-122.372665,9,False
58059,67969787,Signal,11675,9266258,Crown Hill,NaN,False,231081,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,28c0b06a-5d3f-4fb8-8341-575c1859f08a,POINT (-122.37267 47.69656),47.696560,-122.372665,7,False
58060,67969783,Signal,11675,9266258,Crown Hill,NaN,False,231083,PlClKXA8ouVEUv33dO0DAA,6.0000,...,0,0,[ ],None,28c0b06a-5d3f-4fb8-8341-575c1859f08a,POINT (-122.37266 47.69650),47.696499,-122.372658,4,False


In [41]:
#export to geojson
clusters_q1.to_file("data/clusters_q1.geojson", driver='GeoJSON')